### Importación de librerías

In [399]:
import pandas as pd
import numpy as np
import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

In [400]:
import time
import datetime
from datetime import datetime

### Importación de los archivos

In [401]:
df_programas = pd.read_excel("Programas_Tec.xlsx")
print("Dimensiones de df_programas:", df_programas.shape)
df_programas.head(3)

Dimensiones de df_programas: (363, 4)


,Clave,Programa Academico,Area Academica,Escuela
0,AMC19,Ambiente Construido Exploración,Ambiente Construido,"Arquitectura, Arte y Diseño"
1,ARQ,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"
2,ARQ07,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"


In [402]:
df_solicitudes = pd.read_csv("Tablero_solicitudes_resumen.csv")
print("Dimensiones de df_solicitudes:", df_solicitudes.shape)
df_solicitudes.head(3)

Dimensiones de df_solicitudes: (81356, 22)


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Periodo Solicitud,OportunidadesSeleccionadas,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Enero - Mayo 2015,1 - EUA-5INT-020V,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Enero - Mayo 2015,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...","SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Enero - Mayo 2015,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...","CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM


### Limpieza de la base de datos

##### Eliminación valores nulos

In [403]:
df_shape_inicial = df_solicitudes.shape
df_solicitudes.dropna(subset=['OportunidadesSeleccionadas'], inplace=True)
df_solicitudes.dropna(subset=['OportunidadAsignada'], inplace=True)
df_shape_final = df_solicitudes.shape
print("Tamaño antes de eliminar Nan:",df_shape_inicial)
print("Tamaño después de eliminar Nan:",df_shape_final)

Tamaño antes de eliminar Nan: (81356, 22)
Tamaño después de eliminar Nan: (55222, 22)


##### Selección de Estatus válidos

In [404]:
# Valores válidos para Estatus con la subcadena Asignada/Asignado
valid_values = []
status_values = list(df_solicitudes["Estatus"].unique())
for val in status_values:
    if "Asignada" in val:
        valid_values.append(val)
    elif "Asignado" in val:
        valid_values.append(val)

# Eliminación del valor con Estatus Pre-Asignada
valid_values.remove('Pre-Asignada - Pendiente pago')

df = df_solicitudes[df_solicitudes["Estatus"].isin(valid_values)]
print("Dimensiones de df", df.shape)
df.head(3)

Dimensiones de df (31325, 22)


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Periodo Solicitud,OportunidadesSeleccionadas,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Enero - Mayo 2015,1 - EUA-5INT-020V,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Enero - Mayo 2015,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...","SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Enero - Mayo 2015,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...","CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM


##### Identificación de las primeras opciones seleccionadas

In [405]:
df['1era opción seleccionada'] = df['OportunidadesSeleccionadas'].str.split('1 - ').str[1]
df['1era opción seleccionada'] = df['1era opción seleccionada'].str.split(',').str[0]

df['2da opción seleccionada'] = df['OportunidadesSeleccionadas'].str.split('2 - ').str[1]
df['2da opción seleccionada'] = df['2da opción seleccionada'].str.split(',').str[0]

df['3era opción seleccionada'] = df['OportunidadesSeleccionadas'].str.split('3 - ').str[1]
df['3era opción seleccionada'] = df['3era opción seleccionada'].str.split(',').str[0]

df['1era opción seleccionada'] = df['1era opción seleccionada'].fillna("-")
df['2da opción seleccionada'] = df['2da opción seleccionada'].fillna("-")
df['3era opción seleccionada'] = df['3era opción seleccionada'].fillna("-")

print("Dimensiones de df", df.shape)
df.head(3)

C:\Users\hp\AppData\Local\Temp\ipykernel_7276\2914502707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1era opción seleccionada'] = df['OportunidadesSeleccionadas'].str.split('1 - ').str[1]
C:\Users\hp\AppData\Local\Temp\ipykernel_7276\2914502707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1era opción seleccionada'] = df['1era opción seleccionada'].str.split(',').str[0]
C:\Users\hp\AppData\Local\Temp\ipykernel_7276\2914502707.py:4: SettingWithCopyWarning: 
A value is trying to be set o

Dimensiones de df (31325, 25)


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha,1era opción seleccionada,2da opción seleccionada,3era opción seleccionada
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM,EUA-5INT-020V,-,-
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM,SIN-5INT-003,JAP-5INT-002A,CHN-5INT-003
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM,CZE-5INT-001,FIN-5INT-004A,FIN-5INT-024


In [406]:
df = df[['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', '1era opción seleccionada',
       '2da opción seleccionada','3era opción seleccionada',
       'OportunidadesAprobadas', 'Estatus','OportunidadAsignada', 
       'Nombre Oportunidad Asignada','CampusAdministrador', 
       'PeriodoAcadémico', 'Actividad Actual', 'Fecha']]
df.reset_index(drop=True, inplace = True)
df.head(3)

,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,2da opción seleccionada,3era opción seleccionada,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,-,-,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,JAP-5INT-002A,CHN-5INT-003,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,FIN-5INT-004A,FIN-5INT-024,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM


In [407]:
values_1 = list(df['1era opción seleccionada'])
values_2 = list(df['2da opción seleccionada'])
values_3 = list(df['3era opción seleccionada'])

values_opcion_asignada = list(df['OportunidadAsignada'])
Estatus_asignacion = []

for j in range(df.shape[0]):
    if values_1[j] == values_opcion_asignada[j]:
        Estatus_asignacion.append("Asignación a la primera opción")
    elif values_2[j] == values_opcion_asignada[j]:
        Estatus_asignacion.append("Asignación a la segunda opción")
    elif values_3[j] == values_opcion_asignada[j]:
        Estatus_asignacion.append("Asignación a la tercera opción")
    else:
        Estatus_asignacion.append("No asignado a algunas de sus tres opción")
df["Estatus de asignacion"] = Estatus_asignacion

df = df[['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', '1era opción seleccionada',
       '2da opción seleccionada','3era opción seleccionada','Estatus de asignacion',
       'OportunidadesAprobadas', 'Estatus','OportunidadAsignada',
       'Nombre Oportunidad Asignada','CampusAdministrador', 
       'PeriodoAcadémico', 'Actividad Actual', 'Fecha']]
df.head(3)

,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,3era opción seleccionada,Estatus de asignacion,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,-,Asignación a la primera opción,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,CHN-5INT-003,Asignación a la segunda opción,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,FIN-5INT-024,Asignación a la primera opción,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM


In [408]:
pd.DataFrame(df.groupby(['Estatus de asignacion'])['Estatus de asignacion'].count())

,Estatus de asignacion
Estatus de asignacion,
Asignación a la primera opción,28993
Asignación a la segunda opción,830
Asignación a la tercera opción,454
No asignado a algunas de sus tres opción,1048


#### Tipo de intercambio

In [409]:
df['Tipo de intercambio [aux]'] = df['OportunidadAsignada'].str.split('-').str[1]
df['Tipo de intercambio [aux]'] = df['Tipo de intercambio [aux]'].str.split('-').str[0]
print(df.shape)
df.head(3)

(31325, 27)


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Estatus de asignacion,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha,Tipo de intercambio [aux]
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Asignación a la primera opción,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM,5INT
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Asignación a la segunda opción,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM,5INT
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Asignación a la primera opción,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM,5INT


In [410]:
df['Tipo de intercambio'] = pd.np.where(df['Tipo de intercambio [aux]'].str.contains("INT"), "Intercambio Tradicional", "Study Abroad")
df.drop(['Tipo de intercambio [aux]'], axis=1, inplace=True)
df.head(3)


C:\Users\hp\AppData\Local\Temp\ipykernel_7276\1552165655.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df['Tipo de intercambio'] = pd.np.where(df['Tipo de intercambio [aux]'].str.contains("INT"), "Intercambio Tradicional", "Study Abroad")


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Estatus de asignacion,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha,Tipo de intercambio
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Asignación a la primera opción,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM,Intercambio Tradicional
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Asignación a la segunda opción,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM,Intercambio Tradicional
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Asignación a la primera opción,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM,Intercambio Tradicional


#### Obtención de los países destino

In [411]:
df['País destino'] = df['OportunidadAsignada'].str.split('-').str[0]
print(df.shape)
df.head(3)

(31325, 28)


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha,Tipo de intercambio,País destino
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM,Intercambio Tradicional,EUA
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM,Intercambio Tradicional,JAP
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM,Intercambio Tradicional,CZE


In [412]:
df.dropna(subset=["Campus"], inplace=True)
df.shape[0]

30908

#### Merge con la base de datos de los países

In [413]:
df_aux = pd.merge(df, df_programas, how='left', left_on='Programa', right_on='Clave')
df_aux.head(3)

,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha,Tipo de intercambio,País destino,Clave,Programa Academico,Area Academica,Escuela
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM,Intercambio Tradicional,EUA,INT11,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM,Intercambio Tradicional,JAP,ITC11,Ingeniería en Tecnologías Computacionales,Ingeniería y Ciencias,Ingeniería y Ciencias
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM,Intercambio Tradicional,CZE,ISC11,Ingeniería en Sistemas Computacionales,Ingeniería y Ciencias,Ingeniería y Ciencias


In [414]:
pd.DataFrame(df_aux.groupby(["Escuela"])["Escuela"].count()).sort_index()

,Escuela
Escuela,
"Arquitectura, Arte y Diseño",1407
Ciencias Sociales y Gobierno,2429
Humanidades y Educación,1682
Ingeniería y Ciencias,11880
Internacionalización,4
Negocios,10007
No Aplica,83
Prepa Tec,1771
Salud,1641


In [415]:
df_aux = df_aux[['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Programa Academico',
       'Area Academica', 'Escuela','Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', '1era opción seleccionada',
       '2da opción seleccionada', '3era opción seleccionada',
       'Estatus de asignacion', 'OportunidadesAprobadas', 'Estatus',
       'OportunidadAsignada', 'Nombre Oportunidad Asignada','Tipo de intercambio', 'País destino',
       'CampusAdministrador', 'PeriodoAcadémico', 'Actividad Actual', 'Fecha']]

df_aux.head(3)

,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Programa Academico,Area Academica,Escuela,...,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,Tipo de intercambio,País destino,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,...,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Intercambio Tradicional,EUA,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,Ingeniería en Tecnologías Computacionales,Ingeniería y Ciencias,Ingeniería y Ciencias,...,"SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Intercambio Tradicional,JAP,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,Ingeniería en Sistemas Computacionales,Ingeniería y Ciencias,Ingeniería y Ciencias,...,"CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Intercambio Tradicional,CZE,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM


In [416]:
df = df_aux.copy()

for j in range(df.shape[0]):
    if "ENE" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('ENE', '01')
    elif "Ene" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('Ene', '01')

    elif "FEB" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('FEB', '02')
    elif "Feb" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Feb', '02')

    elif "MAR" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('MAR', '03')
    elif "Mar" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Mar', '03')

    elif "ABR" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('ABR', '04')
    elif "Abr" in df["Fecha"][j]:   
        df["Fecha"][j] = df["Fecha"][j].replace('Abr', '04')

    elif "MAY" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('MAY', '05')
    elif "May" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('May', '05')

    elif "JUN" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('JUN', '06')
    elif "Jun" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Jun', '06')

    elif "JUL" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('JUN', '07')
    elif "Jul" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Jul', '07')

    elif "AGO" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('AGO', '08')
    elif "Ago" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Ago', '08')

    elif "SEP" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('SEP', '09')
    elif "Sep" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Sep', '09')

    elif "OCT" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('OCT', '10')
    elif "Oct" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Oct', '10')

    elif "NOV" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('NOV', '11')
    elif "Nov" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Nov', '11')

    elif "DEC" in df["Fecha"][j]:
        df["Fecha"][j] = df["Fecha"][j].replace('DEC', '12')
    elif "Dec" in df["Fecha"][j]:    
        df["Fecha"][j] = df["Fecha"][j].replace('Dec', '12')

C:\Users\hp\AppData\Local\Temp\ipykernel_7276\1555987857.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fecha"][j] = df["Fecha"][j].replace('AGO', '08')
C:\Users\hp\AppData\Local\Temp\ipykernel_7276\1555987857.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fecha"][j] = df["Fecha"][j].replace('Sep', '09')
C:\Users\hp\AppData\Local\Temp\ipykernel_7276\1555987857.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[

In [417]:
años = []
meses = []
dias = []

format_1 = "%d/%m/%Y %H:%M %p"

for i in range(df.shape[0]):
    a = df["Fecha"][i]
    if isinstance(a, str):
        try:
            criter_format = bool(datetime.strptime(a, format_1))
        except ValueError:
            criter_format = False
        if criter_format == True:
            value = datetime.strptime(a,"%d/%m/%Y %H:%M %p")
            años.append(value.year)
            meses.append(value.month)
            dias.append(value.day)
        else:
            value = datetime.strptime(a,"%d/%m/%Y %H:%M%p")
            años.append(value.year)
            meses.append(value.month)
            dias.append(value.day)
    else:
        años.append(a.year)
        meses.append(a.month)
        dias.append(a.day)
df["Año"] = años
df["Mes"] = meses
df["Dia"] = dias
df = df[['Instancia', 'Fecha', 'Año','Mes', 'Dia','Matrícula', 'Nombre' ,'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Programa Academico',
       'Area Academica', 'Escuela','Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud', '1era opción seleccionada',
       '2da opción seleccionada', '3era opción seleccionada',
       'Estatus de asignacion', 'Estatus',
       'OportunidadAsignada', 'Nombre Oportunidad Asignada','Tipo de intercambio', 'País destino',
       'CampusAdministrador', 'PeriodoAcadémico', 'Actividad Actual']]
df.head(3)

,Instancia,Fecha,Año,Mes,Dia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,...,3era opción seleccionada,Estatus de asignacion,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,Tipo de intercambio,País destino,CampusAdministrador,PeriodoAcadémico,Actividad Actual
0,8453,20/08/2014 02:22PM,2014,8,20,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,...,-,Asignación a la primera opción,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Intercambio Tradicional,EUA,Querétaro,Enero - Mayo 2015,Resultado de Asignación
1,8289,29/08/2014 04:34PM,2014,8,29,A01018302,Pablo,Zebadúa,Vega,Santa Fe,...,CHN-5INT-003,Asignación a la segunda opción,Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Intercambio Tradicional,JAP,Monterrey,Enero - Mayo 2015,Resultado de Asignación
2,8510,29/08/2014 04:35PM,2014,8,29,A01205500,Laura,Treviño,Hernández,Querétaro,...,FIN-5INT-024,Asignación a la primera opción,Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Intercambio Tradicional,CZE,Monterrey,Enero - Mayo 2015,Resultado de Asignación


In [418]:
list_cod_paises = ['EUA', 'JAP', 'HOL', 'CRO', 'ALE', 'COR', 'SUI', 'DIN', 'SUE',
       'ING', 'POR', 'SIN', 'SER', 'TAI', 'CHI', 'CRC', 'URU', 'EAU',
       'PRC', 'SAF']

list_cod_iso3 = ['USA', 'JPN', 'NLD', 'HRV', 'DEU', 'KOR', 'CHE', 'DNK', 'SWE',
       'GBR', 'PRT', 'SGP', 'SRB', 'THA', 'CHN', 'CRI','URY','ARE',
       'PRI', 'ZAF']
diccionario_paises = dict(zip(list_cod_paises, list_cod_iso3 ))
diccionario_paises

df['País destino'] = df['País destino'].replace(diccionario_paises)
df.head(3)

,Instancia,Fecha,Año,Mes,Dia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,...,3era opción seleccionada,Estatus de asignacion,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,Tipo de intercambio,País destino,CampusAdministrador,PeriodoAcadémico,Actividad Actual
0,8453,20/08/2014 02:22PM,2014,8,20,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,...,-,Asignación a la primera opción,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Intercambio Tradicional,USA,Querétaro,Enero - Mayo 2015,Resultado de Asignación
1,8289,29/08/2014 04:34PM,2014,8,29,A01018302,Pablo,Zebadúa,Vega,Santa Fe,...,CHN-5INT-003,Asignación a la segunda opción,Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Intercambio Tradicional,JPN,Monterrey,Enero - Mayo 2015,Resultado de Asignación
2,8510,29/08/2014 04:35PM,2014,8,29,A01205500,Laura,Treviño,Hernández,Querétaro,...,FIN-5INT-024,Asignación a la primera opción,Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Intercambio Tradicional,CZE,Monterrey,Enero - Mayo 2015,Resultado de Asignación


### Importar base de datos con los códigos de los países, nombres y continentes

In [419]:
df_paises = pd.read_csv("https://gist.githubusercontent.com/kintero/7d1db891401f56256c79/raw/a61f6d0dda82c3f04d2e6e76c3870552ef6cf0c6/paises.csv")
df_paises = df_paises[[" iso3","nombre","continente"]]
df_paises = df_paises.rename(columns={" iso3":"Código del país","nombre":"Nombre del país","continente":"Continente"})
df_paises.head(3)

,Código del país,Nombre del país,Continente
0,AFG,Afganistán,Asia
1,ALB,Albania,Europa
2,DEU,Alemania,Europa


In [420]:
df_aux = pd.merge(df, df_paises, how='left', left_on='País destino', right_on='Código del país')
df_aux.drop(['Código del país'], axis = 1, inplace=True)
df_aux.head(3)

,Instancia,Fecha,Año,Mes,Dia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,...,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,Tipo de intercambio,País destino,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Nombre del país,Continente
0,8453,20/08/2014 02:22PM,2014,8,20,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,...,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Intercambio Tradicional,USA,Querétaro,Enero - Mayo 2015,Resultado de Asignación,Estados Unidos de América,América
1,8289,29/08/2014 04:34PM,2014,8,29,A01018302,Pablo,Zebadúa,Vega,Santa Fe,...,Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Intercambio Tradicional,JPN,Monterrey,Enero - Mayo 2015,Resultado de Asignación,Japón,Asia
2,8510,29/08/2014 04:35PM,2014,8,29,A01205500,Laura,Treviño,Hernández,Querétaro,...,Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Intercambio Tradicional,CZE,Monterrey,Enero - Mayo 2015,Resultado de Asignación,República Checa,Europa


In [421]:
df_aux = df_aux[['Fecha', 'Año', 'Mes', 'Dia', 'Matrícula', 'Nombre',
       'Ap Paterno', 'Ap Materno', 'Campus', 'Programa', 'Programa Academico',
       'Area Academica', 'Escuela', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       '1era opción seleccionada', '2da opción seleccionada',
       '3era opción seleccionada', 'Estatus de asignacion', 'Estatus',
       'OportunidadAsignada', 'Nombre Oportunidad Asignada',
       'Tipo de intercambio', 'CampusAdministrador',
       'País destino', 'Nombre del país','Continente',
       'PeriodoAcadémico', 'Actividad Actual']]
df_aux.head(3)

,Fecha,Año,Mes,Dia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,...,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,Tipo de intercambio,CampusAdministrador,País destino,Nombre del país,Continente,PeriodoAcadémico,Actividad Actual
0,20/08/2014 02:22PM,2014,8,20,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,...,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Intercambio Tradicional,Querétaro,USA,Estados Unidos de América,América,Enero - Mayo 2015,Resultado de Asignación
1,29/08/2014 04:34PM,2014,8,29,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,...,Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Intercambio Tradicional,Monterrey,JPN,Japón,Asia,Enero - Mayo 2015,Resultado de Asignación
2,29/08/2014 04:35PM,2014,8,29,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,...,Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Intercambio Tradicional,Monterrey,CZE,República Checa,Europa,Enero - Mayo 2015,Resultado de Asignación


In [ ]:
df_aux.to_csv("Base de datos_Proyecto final.csv")

In [434]:
apellidos = df.groupby(['Ap Paterno'])['Ap Paterno'].count().to_dict()
df_apellidos = pd.DataFrame([key for key in apellidos.keys()], columns=['Apellidos'])
df_apellidos['Conteo'] = [value for value in apellidos.values()]
df_apellidos = df_apellidos.sort_values(by="Conteo", ascending=False)
df_apellidos

,Apellidos,Conteo
1883,González,771
1750,García,707
2048,Hernández,661
2674,Martínez,610
2525,López,528
...,...,...
1307,De la Colina,1
2979,Musule,1
2980,Muttio,1
1306,De Zulueta,1


In [435]:
apellidos_maternos = df.groupby(['Ap Materno'])['Ap Materno'].count().to_dict()
df_apellidos_maternos = pd.DataFrame([key for key in apellidos_maternos.keys()], columns=['Apellidos maternos'])
df_apellidos_maternos['Conteo'] = [value for value in apellidos_maternos.values()]
df_apellidos_maternos = df_apellidos_maternos.sort_values(by="Conteo", ascending=False)
df_apellidos_maternos

,Apellidos maternos,Conteo
1789,González,812
1681,García,733
2496,Martínez,603
1948,Hernández,571
2377,López,562
...,...,...
2597,Meníndez,1
864,Carús,1
862,Cartín,1
2602,Mercenario,1
